In [1]:
from transformers import  AutoTokenizer, AutoModelForSeq2SeqLM

/home/dimweb/Desktop/deeppavlov/d_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")